In [20]:
#Import necessary packages
import sys
import pandas as pd
import numpy as np
import time
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import seaborn as sns
import random
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.distributions import Gumbel
from typing import List, Dict
from itertools import groupby

#Import necessary functions
sys.path.append("aux_functions")
from functions_datasets_attert import attert_model as DataBase # define what you import as DataBase!
from functions_datasets_attert import validate_samples, manual_train_test_split
from functions_loss_attert import gaussian_distribution, nll_loss
from functions_aux_attert import create_folder, set_random_seed, write_report

Part 1. Initialize information

In [21]:
# paths to access the information
path_entities = #path to import the different locations IDs; .txt 
path_data = # data folder path

# dynamic forcings and target
dynamic_input = ['P', 'ET', 'T', 'Q', 'Sensor_depth']#, 'Sensor_type_A', 'Sensor_type_B', 'Sensor_type_C', 'Sensor_type_X'] # Sensor ID information (i.e. senyor type) is used for single-Gaussian LSTM, but not for three-Gaussian LSTM

target = ['VWC']

# static attributes that will be used
static_input = ['dem',
       'landuse_A', 'landuse_B', 'landuse_C', 'landuse_D', 'landuse_E',
       'landuse_F', 'soiltype_A', 'soiltype_B', 'soiltype_C', 'soiltype_D']

# time periods
time_period = 2160 # Time period for segmentationg lenght
random_state_separation = 5 # Random seed to ensure different training and testing sets for each initialization


model_hyper_parameters = {
    "input_size": len(dynamic_input) + len(static_input),
    "no_of_layers":1,  
    "seq_length": 128,
    "hidden_size": 64,
    "batch_size": 128,
    "no_of_epochs": 10,             
    "drop_out": 0.4, 
    "learning_rate": 0.0005,
    "adapt_learning_rate_epoch": 1,
    "adapt_gamma_learning_rate": 0.5,
    "set_forget_gate":3
}

# device to train the model
running_device = 'cpu' #cpu or gpu
# define random seed
seed = 42
# Name of the folder where the results will be stored 
path_save_folder = #''

# colorblind friendly palette for plotting
color_palette = {'observed': '#1f78b4','LSTM': '#ff7f00'}

In [22]:
# Create folder to store the results
create_folder(folder_path=path_save_folder)

Folder 'attert_model/saved_epochs/temporal_segmentation/run_12' created successfully.


Part 2. Class to create the dataset object used to manage the information

In [23]:
class BaseDataset(Dataset):

    #Function to initialize the data
    def __init__(self, 
                 dynamic_input: List[str],
                 static_input: List[str],
                 set_type: str,
                 target: List[str], 
                 sequence_length: int,
                 path_entities: str,
                 path_data: str,
                 forcings: List[str] = [],
                 check_NaN:bool = True
                 ):

        # read and create variables
        self.time_period = time_period # time period that is being considere        
        self.dynamic_input = dynamic_input  # dynamic forcings going as inputs of in the lstm
        self.target = target  # target variable
        self.sequence_length = sequence_length # sequence length
        entities_ids = np.loadtxt(path_entities, dtype='str').tolist() 
        
        # save the cathments as a list even if there is just one
        self.entities_ids = [entities_ids] if isinstance(entities_ids, str) else entities_ids # catchments
        
        self.sequence_data = {} # store information that will be used to run the lstm
        self.df_ts = {} # store processed dataframes for all basins
        self.scaler = {} # information to standardize the data 
        self.location_std = {} # std of the target variable of each basin (can be used later in the loss function)
        self.valid_entities= [] # list of the elements that meet the criteria to be used by the lstm
        self.set_type = set_type
        # Process the attributes
        self.static_input = static_input # static attributes going as inputs to the lstm
        if static_input:
            self.df_attributes = self._load_attributes(path_data)


        for id_sensordepth in self.entities_ids:
            # Read files

            df_ts = self._load_data(path_data=path_data, location_id=id_sensordepth, forcings=forcings)
             # Calculate the number of x (selected) - hour elements
            num_segments = len(df_ts) // time_period
             # Create segments of selected hours
            segments = [df_ts.iloc[i * time_period:(i + 1) * time_period].copy() for i in range(num_segments)]

            # Add ID_sep column to each segment (it will be beneficial to be able to make a difference between segments)
            for i, segment in enumerate(segments):
                    segment['ID_sep'] = f"{segment['ID_sensordepth'].iloc[0]}_{i+1}"

            # Example usage:
            all_segments, fixed_test_segments = manual_train_test_split(segments, test_size=0.4, random_state=random_state_separation)

            # Now process the data based on set_type
            if set_type == "train":
                segments = all_segments
            elif set_type == "test":
                segments = fixed_test_segments
            else:
                raise ValueError("Invalid set_type. Allowed values are 'train', 'validation', or 'test'.")

            for seg in segments:
                id_sep = seg['ID_sep'].iloc[0]
                id = seg['ID_sensordepth'].iloc[0]
                df_ts = seg

                # Checks for invalid samples due to NaN or insufficient sequence length
                flag = validate_samples(x=df_ts.loc[:, self.dynamic_input].values, 
                                            y=df_ts.loc[:, self.target].values, 
                                            attributes=self.df_attributes.loc[id].values if self.static_input else None, 
                                            seq_length=self.sequence_length,
                                             check_NaN=check_NaN)
            
                # Create a list that contains the indexes (basin, day) of the valid samples
                valid_samples = np.argwhere(flag == 1)

                self.valid_entities.extend([(id_sep, int(f[0])) for f in valid_samples])
            
                # Only store data if this basin has at least one valid sample in the given period
                if valid_samples.size > 0:
                    self.df_ts[id_sep] = df_ts
                
                    # Create dictionary entry for the basin
                    self.sequence_data[id_sep] = {}

                    # Store the information of the basin in a nested dictionary
                    self.sequence_data[id_sep]['x_d'] = torch.tensor(df_ts.loc[:, self.dynamic_input].values, dtype=torch.float32)
                    self.sequence_data[id_sep]['y'] = torch.tensor(df_ts.loc[:, self.target].values, dtype=torch.float32)
                    if self.static_input:
                        self.sequence_data[id_sep]['x_s'] = torch.tensor(self.df_attributes.loc[id].values, dtype=torch.float32)


               
    def __len__(self):
        return len(self.valid_entities)
    
    def __getitem__(self, id_sensor):
        """This function is used by PyTorch's dataloader to extract the information"""
        location, i = self.valid_entities[id_sensor]

        # tensor of inputs
        x_LSTM = self.sequence_data[location]['x_d'][i-self.sequence_length+1:i+1, :]
        if self.static_input:
            x_s = self.sequence_data[location]['x_s'].repeat(x_LSTM.shape[0],1)
            x_LSTM = torch.cat([x_LSTM, x_s], dim=1)
        
        # tensor of outputs
        y_obs = self.sequence_data[location]['y'][i]

        # optional also return the basin_std
        if self.location_std:
            return x_LSTM, y_obs, self.location_std[location].unsqueeze(0)
        else:
            return x_LSTM, y_obs

    def _load_attributes(self, path_data: str) -> pd.DataFrame:
        """Call the specific function that reads the static attributes information.
        
        Parameters
        ----------
        path_data : str
            path to the folder were the data is stored
            
        Returns
        -------
        df_attributes: pd.DataFrame
            Dataframe containing the attributes of interest for the catchments of interest
        """
        df_attributes = DataBase.read_attributes(path_data=path_data)
        df_attributes = df_attributes.loc[self.entities_ids, self.static_input]
        return df_attributes


    def _load_data(self, path_data: str, location_id:str, forcings:List[str]) -> pd.DataFrame:
        """Call the specific function that reads a specific location timeseries into a dataframe.

        Parameters
        ----------
        path_data : str
            path to the folder were the data is stored.
        location_id : str
            location_id.
        forcings : str

        Returns
        -------
        df: pd.DataFrame
            Dataframe with the locations` timeseries
        """
        df_ts = DataBase.read_data(path_data=path_data, location_id=location_id, forcings = forcings)
        return df_ts


    def calculate_location_std(self):
        """Fill the self.location_std dictionary with the standard deviation of the target variables for each location"""
        for id, data in self.sequence_data.items():
            self.location_std[id] = torch.tensor(np.nanstd(data['y'].numpy()), dtype=torch.float32)
    
    def calculate_global_statistics(self):
        """Fill the self.scalar dictionary 
        
        The function calculates the global mean and standard deviation of the dynamic inputs, target variables and 
        static attributes, and store the in a dictionary. It will be used later to standardize used in the LSTM. This
        function should be called only in training period. 
        """
        global_x = np.vstack([df.loc[:, self.dynamic_input].values for df in self.df_ts.values()])
        self.scaler['x_d_mean'] = torch.tensor(np.nanmean(global_x, axis=0), dtype=torch.float32)
        self.scaler['x_d_std'] = torch.tensor(np.nanstd(global_x, axis=0), dtype=torch.float32)
        del global_x

        global_y = np.vstack([df.loc[:, self.target].values for df in self.df_ts.values()])
        self.scaler['y_mean'] = torch.tensor(np.nanmean(global_y, axis=0), dtype=torch.float32)
        self.scaler['y_std'] = torch.tensor(np.nanstd(global_y, axis=0), dtype=torch.float32)
        del global_y

        if self.static_input:
            self.scaler['x_s_mean'] = torch.tensor(self.df_attributes.mean().values, dtype= torch.float32)
            self.scaler['x_s_std'] = torch.tensor(self.df_attributes.std().values, dtype= torch.float32)
    
    def standardize_data(self, standardize_output:bool=True):
        """Standardize the data used in the LSTM. 

        The function standardize the data contained in the self.sequence_data dictionary 
        
        Parameters
        ----------
        standardize_output : bool
            Boolean to define if the output should be standardize or not. 
        """
        for location in self.sequence_data.values():
            # Standardize input
            location['x_d'] = (location['x_d'] - self.scaler['x_d_mean']) / (self.scaler['x_d_std'])
            if self.static_input:
                location['x_s'] = (location['x_s'] - self.scaler['x_s_mean']) / (self.scaler['x_s_std'])
            # Standardize output
            if standardize_output:
                location['y'] = (location['y'] - self.scaler['y_mean']) / (self.scaler['y_std'])

Part 3. Create the different datasets

In [24]:
# Dataset training
training_dataset = BaseDataset(dynamic_input=dynamic_input,
                               static_input=static_input,
                               target=target,
                               sequence_length=model_hyper_parameters['seq_length'],
                               path_entities=path_entities,
                               set_type='train',
                               path_data=path_data,
                               check_NaN=True)

training_dataset.calculate_global_statistics() # the global statistics are calculated in the training period!
training_dataset.standardize_data()

In [25]:
valid_locations_train = [next(group)[0] for key, group in groupby(training_dataset.valid_entities, key=lambda x: x[0])]

In [27]:
# Dataset validation
validation_dataset = BaseDataset(dynamic_input=dynamic_input,
                                 static_input=static_input,
                                 target=target,
                                 sequence_length=model_hyper_parameters['seq_length'],
                                 path_entities=path_entities,
                                 set_type='train',
                                 path_data=path_data,
                                 check_NaN=False)

validation_dataset.scaler = training_dataset.scaler # read the global statisctics calculated in the training period
validation_dataset.standardize_data(standardize_output=False)

Part 4. Create the different dataloaders

In [28]:
# DataLoader for training data.
train_loader = DataLoader(training_dataset, 
                          batch_size=model_hyper_parameters['batch_size'],
                          shuffle=True,
                          drop_last = True)

print('Batches in training: ', len(train_loader))
x_lstm, y = next(iter(train_loader))
print(f'x_lstm: {x_lstm.shape} | y: {y.shape}')

Batches in training:  21680
x_lstm: torch.Size([128, 128, 16]) | y: torch.Size([128, 1])


In [29]:
# DataLoader for validation data.
validation_batches=[[index for index, _ in group] for _ , group in groupby(enumerate(validation_dataset.valid_entities), 
                                                                           lambda x: x[1][0])] # each basin is one batch

validation_loader = DataLoader(dataset=validation_dataset,
                               batch_sampler=validation_batches)

# see if the batches are loaded correctly
print('Batches in validation: ', len(validation_loader))
x_lstm, y= next(iter(validation_loader))
print(f'x_lstm: {x_lstm.shape} | y: {y.shape}')

# create some lists with the valid basins and the valid entities per basin that will help later to organize the data
valid_locations = [next(group)[0] for key, group in groupby(validation_dataset.valid_entities, key=lambda x: x[0])]
valid_entity_per_location = [[id for _, id in group] for key, group in groupby(validation_dataset.valid_entities, 
                                                                            key=lambda x: x[0])]

Batches in validation:  1365
x_lstm: torch.Size([2033, 128, 16]) | y: torch.Size([2033, 1])


Part 5. Define GM-LSTM class

In [30]:
# check if model will be run in gpu or cpu
if running_device == 'gpu':
    print(torch.cuda.get_device_name(0))
    device= f'cuda:0'
elif running_device == 'cpu':
    device = "cpu"

class Cuda_MDN_LSTM(nn.Module):
    def __init__(self, model_hyper_parameters, num_mixtures=3):
        super().__init__()
        self.num_features = model_hyper_parameters['input_size']
        self.hidden_units = model_hyper_parameters['hidden_size']
        self.num_layers = model_hyper_parameters['no_of_layers']
        self.num_mixtures = num_mixtures

        self.lstm = nn.LSTM(input_size=model_hyper_parameters['input_size'], 
                            hidden_size=model_hyper_parameters['hidden_size'], 
                            batch_first=True,
                            num_layers=model_hyper_parameters['no_of_layers'])

        self.dropout = torch.nn.Dropout(model_hyper_parameters['drop_out'])

        
        self.linear_pi = nn.Linear(in_features=model_hyper_parameters['hidden_size'], out_features=num_mixtures)
        self.linear_mu = nn.Linear(in_features=model_hyper_parameters['hidden_size'], out_features=num_mixtures)
        self.linear_sigma = nn.Linear(in_features=model_hyper_parameters['hidden_size'], out_features=num_mixtures)
           
    def forward(self, x):
        # initialize hidden state with zeros
        batch_size = x.shape[0]
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_units, 
                         requires_grad=True, dtype=torch.float32, device=x.device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_units, 
                         requires_grad=True, dtype=torch.float32, device=x.device)
        
        out, (hn_1, cn_1) = self.lstm(x, (h0, c0))
        out = out[:, -1, :]  # sequence to one
        out = self.dropout(out)

        pi = F.softmax(self.linear_pi(out), dim=-1)  # Gumbel softmax
        
        mu = self.linear_mu(out) # Calculating the means of the Gaussian distributions. The linear transformation maps the output to the space of mean values for each Gaussian component
        sigma = torch.exp(self.linear_sigma(out))  # Computing standard deviations of the Gaussian distributions. Using exponential to ensure positivity - requirement for valid Gaussian distributions
        
        return pi, mu, sigma

In [31]:
# Normalization factor
'''To ensure that the total probability under the distribution integrates to 1'''
oneDivSqrtTwoPI = 1.0 / np.sqrt(2.0*np.pi)

Part 6. Construct and run GM-LSTM model

In [32]:
# Construct model
set_random_seed(seed=seed)
lstm_model = Cuda_MDN_LSTM(model_hyper_parameters).to(device)

# Optimizer
optimizer = torch.optim.Adam(lstm_model.parameters(),
                             lr=model_hyper_parameters["learning_rate"])

# Define learning rate scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                            step_size=model_hyper_parameters["adapt_learning_rate_epoch"],
                                            gamma=model_hyper_parameters["adapt_gamma_learning_rate"])

# Set forget gate to ensure long-term dependency learning
lstm_model.lstm.bias_hh_l0.data[model_hyper_parameters['hidden_size']:2 * model_hyper_parameters['hidden_size']] = model_hyper_parameters["set_forget_gate"]

training_time = time.time()

for epoch in range(1, model_hyper_parameters["no_of_epochs"] + 1):
    epoch_start_time = time.time()
    total_loss = []  # List to store all training losses for the current epoch

    # Training
    lstm_model.train()
    for x_lstm, y in train_loader:
        optimizer.zero_grad()  # Reset gradients

        # Forward pass
        pi, mu, sigma = lstm_model(x_lstm.to(device).float())

        # Calculate loss
        loss = nll_loss(pi, mu, sigma, y.to(device).float())

        # Backpropagation and optimization
        loss.backward()
        optimizer.step()

        # Add current batch loss to total_loss
        total_loss.append(loss.item())

        # Clear memory
        del y, loss
        torch.cuda.empty_cache()

    # Calculate average loss for the epoch
    average_loss_training = sum(total_loss) / len(train_loader)

    # Save model, optimizer, and scheduler states after each epoch
    path_saved_model = f'{path_save_folder}/epoch_{epoch}'
    torch.save(lstm_model.state_dict(), path_saved_model)
    torch.save(optimizer.state_dict(), f'{path_save_folder}/optimizer_epoch_{epoch}.pt')
    torch.save(scheduler.state_dict(), f'{path_save_folder}/scheduler_epoch_{epoch}.pt')

    # Print epoch report
    epoch_training_time = time.time() - epoch_start_time
    LR = optimizer.param_groups[0]['lr']
    report = (f'Epoch: {epoch:<2} | Loss training: {"%.3f" % average_loss_training} | '
              f'LR: {"%.5f" % LR} | Training time: {"%.1f" % epoch_training_time} s')
    print(report)

    # Save epoch report in txt file
    write_report(file_path=f'{path_save_folder}/run_progress.txt', text=report)

    # Update learning rate
    scheduler.step()

# Print total report
total_training_time = time.time() - training_time
report = f'Total training time: {"%.1f" % total_training_time} s'
print(report)
write_report(file_path=f'{path_save_folder}/run_progress.txt', text=report)

Epoch: 1  | Loss training: 0.682 | LR: 0.00050 | Training time: 1190.3 s
Epoch: 2  | Loss training: 0.497 | LR: 0.00025 | Training time: 1133.4 s
Epoch: 3  | Loss training: 0.441 | LR: 0.00013 | Training time: 1134.9 s
Epoch: 4  | Loss training: 0.416 | LR: 0.00006 | Training time: 1180.4 s
Epoch: 5  | Loss training: 0.404 | LR: 0.00003 | Training time: 1170.1 s
Total training time: 5809.1 s


Part 7. Test GM-LSTM

In [33]:
# In case I already trained an LSTM I can re-construct the model
#path_save_folder = 'attert_model/saved_epochs/temporal_segmentation/run_12/'
#lstm_model = Cuda_MDN_LSTM(model_hyper_parameters).to(device)
#lstm_model.load_state_dict(torch.load(path_save_folder + '/epoch_5'))

<All keys matched successfully>

In [34]:
# Dataset testing
test_dataset = BaseDataset(dynamic_input=dynamic_input,
                           static_input=static_input,
                           target=target,
                           sequence_length=model_hyper_parameters['seq_length'],
                           set_type='test',
                           path_entities=path_entities,
                           path_data=path_data,
                           check_NaN=False)

test_dataset.scaler = training_dataset.scaler # read the global statisctics calculated in the training period
test_dataset.standardize_data(standardize_output=False)

# DataLoader for testing data.
test_batches=[[index for index, _ in group] for _ , group in groupby(enumerate(test_dataset.valid_entities), 
                                                                     lambda x: x[1][0])]

test_loader = DataLoader(dataset=test_dataset, batch_sampler=test_batches)

# see if the batches are loaded correctly
print('Batches in testing: ', len(test_loader))
x_lstm, y= next(iter(test_loader))
print(f'x_lstm: {x_lstm.shape} | y: {y.shape}')

# create some lists with the valid basins and the valid entities per basin that will help later to organize the data
valid_location_testing = [next(group)[0] for key, group in groupby(test_dataset.valid_entities, key=lambda x: x[0])]
valid_entity_per_location_testing = [[id for _, id in group] for key, group in groupby(test_dataset.valid_entities, 
                                                                                    key=lambda x: x[0])]

Batches in testing:  687
x_lstm: torch.Size([2033, 128, 16]) | y: torch.Size([2033, 1])


In [35]:
# Double check if there are common items (i.e. location segments in training and testing sets)
def find_common_items(list1, list2):
    # Find common items using set intersection
    common_items = set(list1) & set(list2)
    if common_items:
        print(f"Common items found: {common_items}")
    else:
        print("No common items found.")


# Check for common items
find_common_items(valid_location_testing, valid_locations_train)

No common items found.


In [36]:
# Initialize lists to store pi, mu, sigma for all locations
pi_values_list = []
mu_values_list = []
sigma_values_list = []

# Initialize test_results list
test_results = []

with torch.no_grad():
    for i, (x_lstm, y) in enumerate(test_loader):
        # Run LSTM prediction
        pi, mu, sigma = lstm_model(x_lstm.to(device).float())

        # Scale mu and sigma back to the original scale
        mu = mu * test_dataset.scaler['y_std'].to(device) + test_dataset.scaler['y_mean'].to(device)
        sigma = sigma * test_dataset.scaler['y_std'].to(device)

        # Extract corresponding dataframe for the current location/entity
        df_ts = test_dataset.df_ts[valid_location_testing[i]].iloc[valid_entity_per_location_testing[i]]

        # Create a copy of the slice to avoid SettingWithCopyWarning
        df_ts_copy = df_ts.copy().reset_index()

        # Assign pi, mu, sigma values to the copied slice - adjust the number of pi, mu, and sigmas according to number of mixtures used in the GM-LSTM model
        df_ts_copy['pi_1'] = pi[:, 0].cpu().detach().numpy()
        df_ts_copy['pi_2'] = pi[:, 1].cpu().detach().numpy()
        df_ts_copy['pi_3'] = pi[:, 2].cpu().detach().numpy()

        df_ts_copy['mu_1'] = mu[:, 0].cpu().detach().numpy()
        df_ts_copy['mu_2'] = mu[:, 1].cpu().detach().numpy()
        df_ts_copy['mu_3'] = mu[:, 2].cpu().detach().numpy()

        df_ts_copy['sigma_1'] = sigma[:, 0].cpu().detach().numpy()
        df_ts_copy['sigma_2'] = sigma[:, 1].cpu().detach().numpy()
        df_ts_copy['sigma_3'] = sigma[:, 2].cpu().detach().numpy()

        # Store df_ts_copy in test_results list
        test_results.append(df_ts_copy)

        # Store pi, mu, sigma values in lists for later concatenation
        pi_values_list.append(pi.cpu().detach().numpy())
        mu_values_list.append(mu.cpu().detach().numpy())
        sigma_values_list.append(sigma.cpu().detach().numpy())

# Concatenate pi, mu, sigma values across all timesteps
pi_values = np.concatenate(pi_values_list, axis=0)
mu_values = np.concatenate(mu_values_list, axis=0)
sigma_values = np.concatenate(sigma_values_list, axis=0)

# Check shapes for debugging
print(f'Shape of pi_values: {pi_values.shape}')
print(f'Shape of mu_values: {mu_values.shape}')
print(f'Shape of sigma_values: {sigma_values.shape}')

# Convert test_results to a dictionary for easier access
test_results_dict = {valid_location_testing[i]: result for i, result in enumerate(test_results)}

Shape of pi_values: (1396671, 3)
Shape of mu_values: (1396671, 3)
Shape of sigma_values: (1396671, 3)


In [37]:
# Save all test results in a comnbined dataframe
filtered_dfs=[]

# Loop through each location and filter the dataframe
for location, df_ts in test_results_dict.items():
    filtered_df = df_ts[['date', 'VWC', 'P', 'ET', 'T', 'Q', 'ERA5_VWC',
                         'pi_1', 'pi_2', 'pi_3',
                         'sigma_1', 'sigma_2', 'sigma_3',
                         'mu_1', 'mu_2', 'mu_3']].copy() # select any
    
    # Append filtered dataframe to the list
    filtered_dfs.append(filtered_df)

# Concatenate all filtered dataframes into one large dataframe
total_locations_df = pd.concat(filtered_dfs, ignore_index=True).dropna()

# Save the large dataframe to a CSV file
total_locations_df.to_csv('', index=False)

In [38]:
# Filter and save for all different locations (i.e. segments)
filtered_results_dict = {}

for location, df_ts in test_results_dict.items():
    filtered_df = df_ts[['date', 'VWC', 'P', 'ET', 'T', 'Q', 'ERA5_VWC',
                         'pi_1', 'pi_2', 'pi_3',
                         'sigma_1', 'sigma_2', 'sigma_3',
                         'mu_1', 'mu_2', 'mu_3']].copy() # selecty any
    
    # Store filtered dataframe in the new dictionary
    filtered_results_dict[location] = filtered_df

# Save filtered_results_dict to a file (e.g., CSV)
for location, df in filtered_results_dict.items():
    df.to_csv(f'', index=False)